# Helper Code for Assignment 3 (RNN language models)

## Reading raw text file & Create DataLoader

In [ ]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import numpy.ma as ma

In [ ]:
class Vocabulary:

    def __init__(self, pad_token="<pad>", unk_token='<unk>'):
        self.id_to_string = {}
        self.string_to_id = {}

        # add the default pad token
        self.id_to_string[0] = pad_token
        self.string_to_id[pad_token] = 0

        # add the default unknown token
        self.id_to_string[1] = unk_token
        self.string_to_id[unk_token] = 1

        # shortcut access
        self.pad_id = 0
        self.unk_id = 1

    def __len__(self):
        return len(self.id_to_string)

    def add_new_word(self, string):
        self.string_to_id[string] = len(self.string_to_id)
        self.id_to_string[len(self.id_to_string)] = string

    # Given a string, return ID
    def get_idx(self, string, extend_vocab=False):
        if string in self.string_to_id:
            return self.string_to_id[string]
        elif extend_vocab:  # add the new word
            self.add_new_word(string)
            return self.string_to_id[string]
        else:
            return self.unk_id


# Read the raw txt file and generate a 1D PyTorch tensor
# containing the whole text mapped to sequence of token IDs, and a vocab object.
class TextData:

    def __init__(self, file_path, vocab=None, extend_vocab=True, device='cuda'):
        self.data, self.vocab = self.text_to_data(file_path, vocab, extend_vocab, device)

    def __len__(self):
        print(len(self.data))
        return len(self.data)

    def text_to_data(self, text_file, vocab, extend_vocab, device):
        """Read a raw text file and create its tensor and the vocab.

        Args:
          text_file: a path to a raw text file.
          vocab: a Vocab object
          extend_vocab: bool, if True extend the vocab
          device: device

        Returns:
          Tensor representing the input text, vocab file

        """
        assert os.path.exists(text_file)
        if vocab is None:
            vocab = Vocabulary()

        data_list = []
        num_lines=0
        number_chars=0
        capital=0
        lowcase=0


        # Construct data
        full_text = []
        print(f"Reading text file from: {text_file}")
        with open(text_file, 'r') as text:
            for line in text:
                tokens = list(line)
                num_lines+=1
                for token in tokens:
                    number_chars+=1
                    # get index will extend the vocab if the input
                    # token is not yet part of the text.
                    full_text.append(vocab.get_idx(token, extend_vocab=extend_vocab))

                    if token.isupper():
                      capital+=1
                    if token.islower():
                      lowcase+=1
        # convert to tensor
        data = torch.tensor(full_text, device=device, dtype=torch.int64)
        print("Number of lines {}".format(num_lines))
        print("Number of character {}".format(number_chars))
        print("Number of Capital Letters number {}".format(capital))
        print("Number of Lower Letters number {}".format(lowcase))
        print("Number of Unique Character {}".format(len(vocab.string_to_id)))
        print("Done.")

        return data, vocab


# Since there is no need for schuffling the data, we just have to split
# the text data according to the batch size and bptt length.
# The input to be fed to the model will be batch[:-1]
# The target to be used for the loss will be batch[1:]
class DataBatches:

    def __init__(self, data, bsz, bptt_len, pad_id):
        self.batches = self.create_batch(data, bsz, bptt_len, pad_id)

    def __len__(self):
        return len(self.batches)

    def __getitem__(self, idx):
        return self.batches[idx]

    def create_batch(self, input_data, bsz, bptt_len, pad_id):
        """Create batches from a TextData object .

        Args:
          input_data: a TextData object.
          bsz: int, batch size
          bptt_len: int, bptt length
          pad_id: int, ID of the padding token

        Returns:
          List of tensors representing batches

        """
        batches = []  # each element in `batches` is (len, B) tensor
        text_len = len(input_data)
        segment_len = text_len // bsz + 1

        # Question: Explain the next two lines!
        padded = input_data.data.new_full((segment_len * bsz,), pad_id)
        padded[:text_len] = input_data.data
        padded = padded.view(bsz, segment_len).t()
        num_batches = segment_len // bptt_len + 1

        for i in range(num_batches):
            # Prepare batches such that the last symbol of the current batch
            # is the first symbol of the next batch.
            if i == 0:
                # Append a dummy start symbol using pad token
                batch = torch.cat(
                    [padded.new_full((1, bsz), pad_id),
                     padded[i * bptt_len:(i + 1) * bptt_len]], dim=0)
                batches.append(batch)
            else:
                batches.append(padded[i * bptt_len - 1:(i + 1) * bptt_len])

        return batches

In [ ]:
# downlaod the text
# Make sure to go to the link and check how the text looks like.
!wget http://www.gutenberg.org/files/49010/49010-0.txt

--2022-12-04 12:22:49--  http://www.gutenberg.org/files/49010/49010-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/files/49010/49010-0.txt [following]
--2022-12-04 12:22:50--  https://www.gutenberg.org/files/49010/49010-0.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 185303 (181K) [text/plain]
Saving to: ‘49010-0.txt.1’

49010-0.txt.1       100%[===================>] 180.96K   596KB/s    in 0.3s    

2022-12-04 12:22:50 (596 KB/s) - ‘49010-0.txt.1’ saved [185303/185303]



In [ ]:
# This is for Colab. Adapt the path if needed.
text_path = "/content/49010-0.txt"

In [ ]:
DEVICE = 'cuda'

batch_size = 32
bptt_len = 64

my_data = TextData(text_path, device=DEVICE)
batches = DataBatches(my_data, batch_size, bptt_len, pad_id=0)

Reading text file from: /content/49010-0.txt
Number of lines 5033
Number of character 177517
Number of Capital Letters number 9666
Number of Lower Letters number 122224
Number of Unique Character 107
Done.
177517


## Model

In [ ]:
# RNN based language model
class RNNModel(nn.Module):

    def __init__(self, num_classes, emb_dim, hidden_dim, num_layers):
        """Parameters:

          num_classes (int): number of input/output classes
          emb_dim (int): token embedding size
          hidden_dim (int): hidden layer size of RNNs
          num_layers (int): number of RNN layers
        """
        super().__init__()
        self.num_classes = num_classes
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.input_layer = nn.Embedding(num_classes, emb_dim)
        self.rnn = nn.RNN(emb_dim, hidden_dim, num_layers)
        self.out_layer = nn.Linear(hidden_dim, num_classes)

    def forward(self, input, state):
        emb = self.input_layer(input)
        output, state = self.rnn(emb, state)
        output = self.out_layer(output)
        output = output.view(-1, self.num_classes)
        return output, state

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return weight.new_zeros(self.num_layers, bsz, self.hidden_dim)


# To be modified for LSTM...
def custom_detach(h):
    return h.detach()

## Decoding

In [ ]:
@torch.no_grad()
def complete(model, prompt, steps, sample=False):
    """Complete the prompt for as long as given steps using the model.

    Parameters:
      model: language model
      prompt (str): text segment to be completed
      steps (int): number of decoding steps.
      sample (bool): If True, sample from the model. Otherwise greedy.

    Returns:
      completed text (str)
    """
    model.eval()
    out_list = []

    # forward the prompt, compute prompt's ppl
    prompt_list = []
    char_prompt = list(prompt)
    for char in char_prompt:
        prompt_list.append(my_data.vocab.string_to_id[char])
    x = torch.tensor(prompt_list).to(DEVICE).unsqueeze(1)

    states = model.init_hidden(1)
    logits, states = model(x, states)
    probs = F.softmax(logits[-1], dim=-1)

    if sample:
        ix=torch.multinomial(probs, num_samples=1)

    else:
        max_p, ix = torch.topk(probs, k=1, dim=-1)

    out_list.append(my_data.vocab.id_to_string[int(ix)])
    x = ix.unsqueeze(1)

    # decode
    for k in range(steps):
        logits, states = model(x, states)
        probs = F.softmax(logits, dim=-1)
        if sample:  # sample from the distribution or take the most likely
             ix=torch.multinomial(probs, num_samples=1)

        else:
            _, ix = torch.topk(probs, k=1, dim=-1)
        out_list.append(my_data.vocab.id_to_string[int(ix)])
        x = ix
    return ''.join(out_list)

In [ ]:
learning_rate = 0.0005
clipping = 1.0
embedding_size = 64
rnn_size = 2048
rnn_num_layers = 1

# vocab_size = len(module.vocab.itos)
vocab_size = len(my_data.vocab.id_to_string)
print(F"vocab size: {vocab_size}")

model = RNNModel(
    num_classes=vocab_size, emb_dim=embedding_size, hidden_dim=rnn_size,
    num_layers=rnn_num_layers)
model = model.to(DEVICE)
hidden = model.init_hidden(batch_size)

loss_fn = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

vocab size: 107


## Training loop

In [ ]:
# Training

num_epochs = 30
report_every = 30
prompt = "Dogs like best to"
perp_list=[]
gen_text=[]

for ep in range(num_epochs):
    print(f"=== start epoch {ep} ===")
    state = model.init_hidden(batch_size)
    for idx in range(len(batches)):
        batch = batches[idx]
        model.train()
        optimizer.zero_grad()
        state = custom_detach(state)

        input = batch[:-1]
        target = batch[1:].reshape(-1)

        bsz = input.shape[1]
        prev_bsz = state.shape[1]
        if bsz != prev_bsz:
            state = state[:, :bsz, :]
        output, state = model(input, state)
        loss = loss_fn(output, target)
        perp=torch.exp(loss)
        perp_list.append(perp)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clipping)
        optimizer.step()
        if idx % report_every == 0:
            # print(f"train loss: {loss.item()}")  # replace me by the line below!
            print(f"train ppl: {perp}")
            if ((ep==5 or ep==15 or ep==28)and idx==0):
                            generated_text = complete(model, prompt, 128, sample=False)
                            gen_text.append(str(generated_text)+"\n")

            generated_text = complete(model, prompt, 128, sample=False)
            print(f'----------------- epoch/batch {ep}/{idx} -----------------')
            print(prompt)
            print(generated_text)
            print(f'----------------- end generated text -------------------')

=== start epoch 0 ===
train ppl: 107.58854675292969
----------------- epoch/batch 0/0 -----------------
Dogs like best to
                                                                                                                                 
----------------- end generated text -------------------
train ppl: 13.570418357849121
----------------- epoch/batch 0/30 -----------------
Dogs like best to
 the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the 
----------------- end generated text -------------------
train ppl: 11.549098014831543
----------------- epoch/batch 0/60 -----------------
Dogs like best to
 the se the sed the sored the sored the sored the sored the sored the sored the sored the sored the sored the sored the sored the
----------------- end generated text -------------------
=== start epoch 1 ===
train ppl: 10.185006141662598
----------------- epoch/batch 1/0 -----------------
Dogs like b

In [ ]:
print(gen_text)

[' the count a Fox a Fox a Fox a Fox a Fox a Fox a Fox a Fox a Fox a Fox a Fox a Fox a Fox a Fox a Fox a Fox a Fox a Fox a Fox a Fo\n', ' the edge of the most of his hand, and so was all down and see to get a shall be to the end of the most of his hand, and so was a\n', '\ndistinguish that strong and proud.”\n\n\n\n\nTHE WOLF AND THE SHEEP\n\n\nA THE FOX AND THE GOAT\n\n\nAN Ant, which has sprended hard for hi\n']


In [ ]:
perp=[t.to("cpu").detach().numpy() for t in perp_list]
perp=np.stack(perp)

In [ ]:
import pandas as pd
import plotly.express as px
df=pd.DataFrame(perp)
px.scatter(df)


EXO 2.3.A

In [ ]:
print(complete(model, "THE DONKEY IN THE LION’S SKIN", 512, sample=False))




A DONKEY once put on a Lion’s skin which some hunters had spread out
to deserved to chooging all thing
with her. “There is nothing that we want but that this fine weather and
he drew for the
Dovechowed on the world.”

“What I have some yourued my life is nothing. At most it confes of the Hares said to his bell and crow.

1. The moremest required to and bedous of Gromped out to dele his cries, as I well well,” said the Man, “to get them by
ore to a Gardinging betaring of thes were get out, better death to 


EXO 2.3.B

In [ ]:
text=complete(model, "THE DOG IN THE LONE HOUSE", 512, sample=False)
print(text)

 AND THE GOWK


A MOU AND THE LAMB


A WOL Man once her adains, and flowing upon him all other work and you for do intells in
labous neck was the mad in a housekeeping oven the braved, and you have resied, and edented har least horself ourself in its orm to his mother, and you little clance of the world. Let us have heary for the longer.”




THE COCK AND THE COLK


A HUN who had all beciuned in the water, but one soon gets
used himself in the dog sud lowaly for it is not
amusing; and net an all don’t knowne


EXO 2.3.C

In [ ]:
print(complete(model, "The monkey eat ", 512, sample=False))

it. “When the charces as
one of the lobted for a
limbly and procied aroud aty, and electronic work, you into the beants.




THE DOG IN THE LION’S SKIN


A DONKEY once door leaped hardleds asong the coor.

“Hor fie company, and at a holly gear by teres them and fand, and all the more.”




THE COCK AND THE COCK


A HUN The Mouse so wherever in companions, lunging upon him all other work and you for do intells in
labous neck was the mad in a housekeeping oven the braved, and you have resied, and edented har l


EXO 2.3.D

In [ ]:
print(complete(model, "Another bites to ", 512, sample=False))

pooked upon her home. But as he could and cold for a provine.”

The Fowler tree for him as hear of heary at
even my good,” said the Fowler. “Wicked in a house.”

The Fox so much of such a veighbor,” said her young one can only beautiful than the other lived and careful in leapand, and so well heary your veally and ede to cuar one mistaken for the edge of the works, who were deart, and at all the years and not in sight.”

“When the laws of the country road
proud of his trouble.

“Lazy fellow,” said Bersuse





#Model LSTM

In [ ]:
# LSTM based language model
class LSTMModel(nn.Module):

    def __init__(self, num_classes, emb_dim, hidden_dim, num_layers):
        """Parameters:

          num_classes (int): number of input/output classes
          emb_dim (int): token embedding size
          hidden_dim (int): hidden layer size of RNNs
          num_layers (int): number of RNN layers
        """
        super().__init__()
        self.num_classes = num_classes
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.input_layer = nn.Embedding(num_classes, emb_dim,padding_idx=0)
        self.lstm = nn.LSTM(emb_dim, hidden_dim, num_layers)
        self.out_layer = nn.Linear(hidden_dim, num_classes)

    def forward(self, input,state):
        emb = self.input_layer(input)
        output, (hidden_state,cell_state) = self.lstm(emb, state)
        output = output.reshape(output.size(0)*output.size(1), output.size(2))
        output = self.out_layer(output)

        return output, (hidden_state,cell_state)

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return weight.new_zeros(self.num_layers, bsz, self.hidden_dim)

# To be modified for LSTM...
def custom_detach(h):
    return h.detach()


#Decoding LSTM

In [ ]:
@torch.no_grad()
def complete2(model, prompt, steps, sample=False):
    """Complete the prompt for as long as given steps using the model.

    Parameters:
      model: language model
      prompt (str): text segment to be completed
      steps (int): number of decoding steps.
      sample (bool): If True, sample from the model. Otherwise greedy.

    Returns:
      completed text (str)
    """
    model.eval()
    out_list = []

    # forward the prompt, compute prompt's ppl
    prompt_list = []
    char_prompt = list(prompt)
    for char in char_prompt:
        prompt_list.append(my_data.vocab.string_to_id[char])
    x = torch.tensor(prompt_list).to(DEVICE).unsqueeze(1)

    hidden_state = model.init_hidden(1)
    cell_state = model.init_hidden(1)
    logits,(hidden_state,cell_state) = model(x, (hidden_state,cell_state))
    probs = F.softmax(logits[-1], dim=-1)

    if sample:
        ix=torch.multinomial(probs, num_samples=1)
        #assert False
    else:
        max_p, ix = torch.topk(probs, k=1, dim=-1)

    out_list.append(my_data.vocab.id_to_string[int(ix)])
    x = ix.unsqueeze(1)

    # decode
    for k in range(steps):
        logits,(hidden_state,cell_state)= model(x, (hidden_state,cell_state))
        probs = F.softmax(logits, dim=-1)
        if sample:  # sample from the distribution or take the most likely
             ix=torch.multinomial(probs, num_samples=1)
             #assert False
        else:
            _, ix = torch.topk(probs, k=1, dim=-1)
        out_list.append(my_data.vocab.id_to_string[int(ix)])
        x = ix
    return ''.join(out_list)

In [ ]:
learning_rate = 0.001
clipping = 1.0
embedding_size = 64
lstm_size = 2048
lstm_num_layers = 1

# vocab_size = len(module.vocab.itos)
vocab_size = len(my_data.vocab.id_to_string)
print(F"vocab size: {vocab_size}")

model2 = LSTMModel(
    num_classes=vocab_size, emb_dim=embedding_size, hidden_dim=lstm_size,
    num_layers=lstm_num_layers)
model2 = model2.to(DEVICE)
hidden = model2.init_hidden(batch_size)

loss_fn = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = torch.optim.Adam(params=model2.parameters(), lr=learning_rate)

vocab size: 107


In [ ]:
# Training

num_epoch =0
report_every = 30
prompt = "Dogs like best to"
perp=50

while perp > 1.03:
    num_epoch+=1


    print(f"=== start epoch {num_epoch} ===")

    hidden_state = model2.init_hidden(batch_size)
    cell_state = model2.init_hidden(batch_size)
    for idx in range(len(batches)):
        batch = batches[idx]
        model2.train()
        optimizer.zero_grad()

        input = batch[:-1]
        target = batch[1:].flatten()

        output,(hidden_state,cell_state)= model2(input, (hidden_state,cell_state))

        loss = loss_fn(output, target)
        perp=torch.exp(loss)
        perp=perp.to("cpu").detach().numpy()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model2.parameters(), clipping)
        optimizer.step()

        hidden_state=custom_detach(hidden_state)
        cell_state=custom_detach(cell_state)
        if perp < 1.03:
          break
        if idx % report_every == 0:
            # print(f"train loss: {loss.item()}")  # replace me by the line below!
            print(f"train ppl: {perp}")
            generated_text = complete2(model2, prompt, 128, sample=False)
            print(f'----------------- epoch/batch {num_epoch}/{idx} -----------------')
            print(prompt)
            print(generated_text)
            print(f'----------------- end generated text -------------------')

print("final perplexiity : ",perp,"\n")
print("final Epoch :",num_epoch)


=== start epoch 1 ===
train ppl: 106.93266296386719
----------------- epoch/batch 1/0 -----------------
Dogs like best to
                                                                                                                                 
----------------- end generated text -------------------
train ppl: 13.471542358398438
----------------- epoch/batch 1/30 -----------------
Dogs like best to
 the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the 
----------------- end generated text -------------------
train ppl: 10.902223587036133
----------------- epoch/batch 1/60 -----------------
Dogs like best to
 the sed the sed the sed the sed the sed the sed the sed the sed the sed the sed the sed the sed the sed the sed the sed the sed 
----------------- end generated text -------------------
=== start epoch 2 ===
train ppl: 8.557916641235352
----------------- epoch/batch 2/0 -----------------
Dogs like be

#EXO 3.4.A

 Title of a fable which exists in the book.

In [ ]:
print(complete2(model2, "The Fox and the Goat", 512, sample=False))

 feed upon the Lamb, the Wolf left few
of them uninjured.




THE FISHERMAN AND THE LITTLE FISH


ALL day long a Fisherman had been toiling and had caught nothing.

“I will make one more effort,” thought he, “and then I must go home.”

He threw in his line, and soon drew up a very small perch.

The little Fish was terribly frightened when he found himself out of
water, and with a sharp hook sticking in his mouth; and he said to the
Fisherman:

“O sir, take pity upon me, and throw me into the water again! See


In [ ]:
print(complete2(model2, "The Fox and the Goat", 512, sample=True))

 for hunger, ruphing that home, whe house he was thinking how sad that
when he grew no what we are clow
indering that he slow to go what it was impossible, and that he
must die there, a prisoner. While he was thinking how sad that would
be, a thirsty Goat came and looked down intortanity, but mine from a comparison will open the water so
that I cannot drive its best to eat the Lamb, he said, “You little wretch,
if it was not you it was your father, so it’s all the simple creidure that I have spoiled the wate


# EXO 3.4.B
title which invented,is not in the book, but similar in the style.

In [ ]:
print(complete2(model2, "The Serpent and the Cow", 512, sample=False))

'n for her home. But it was not long before they found that he slew more\nof them in a single day than the Kite could possibly pounce upon in a\nwhole year.\n\nThe oldest, wisest pigeon among them said: “When we are in trouble,\nwe must not forget that there are other dangers than the ones we are\nsuffering from. There is a proverb among men that tells them to avoid a\nremedy that is worse than the disease.”\n\n\n\n\nTHE WAR HORSE AND THE MULE\n\n\nA WAR Horse, ready for battle, with his splendid saddle and jingling\nbridle,'

In [ ]:
print(complete2(model2, "The Serpent and the Cow", 512, sample=True))

'n came running home to her child,\ntherefore, that while, for his promise’s sake, he would give her the\npower to harm, she must be careful to him and showed them a bundle of the food, and the Hawk to let her go.\n\n“If you are hungry,” said she, “why not catch some large bird? I am not\nbig enough for even a luncheon.”\n\n“Do you happen to see many large birds flying about?” the Hawk asked. “I should be\nfoolish, indeed, to let you go for the sake of larger birds that are hilding, then I must go home.” So he\nhad en'

#Bonus
Train model with python script

In [ ]:
# This is for Colab. Adapt the path if needed.
text_path = "/content/python.txt"

In [ ]:
DEVICE = 'cuda'

batch_size = 32
bptt_len = 64

my_data = TextData(text_path, device=DEVICE)
batches = DataBatches(my_data, batch_size, bptt_len, pad_id=0)

AssertionError: ignored

In [ ]:
learning_rate = 0.001
clipping = 1.0
embedding_size = 64
lstm_size = 2048
lstm_num_layers = 1

# vocab_size = len(module.vocab.itos)
vocab_size = len(my_data.vocab.id_to_string)
print(F"vocab size: {vocab_size}")

model2 = LSTMModel(
    num_classes=vocab_size, emb_dim=embedding_size, hidden_dim=lstm_size,
    num_layers=lstm_num_layers)
model2 = model2.to(DEVICE)
hidden = model2.init_hidden(batch_size)

loss_fn = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = torch.optim.Adam(params=model2.parameters(), lr=learning_rate)

In [ ]:
# Training

num_epoch =0
report_every = 30
prompt = "Def func "
perp=3
while perp > 1.03:
    num_epoch+=1


    print(f"=== start epoch {num_epoch} ===")

    hidden_state = model2.init_hidden(batch_size)
    cell_state = model2.init_hidden(batch_size)
    for idx in range(len(batches)):
        batch = batches[idx]
        model2.train()
        optimizer.zero_grad()

        input = batch[:-1]
        target = batch[1:].flatten()

        output,(hidden_state,cell_state)= model2(input, (hidden_state,cell_state))

        loss = loss_fn(output, target)
        perp=torch.exp(loss)
        perp=perp.to("cpu").detach().numpy()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model2.parameters(), clipping)
        optimizer.step()

        hidden_state=custom_detach(hidden_state)
        cell_state=custom_detach(cell_state)
        if perp < 1.03:
          break
        if idx % report_every == 0:
            # print(f"train loss: {loss.item()}")  # replace me by the line below!
            print(f"train ppl: {perp}")
            generated_text = complete2(model2, prompt, 128, sample=False)
            print(f'----------------- epoch/batch {num_epoch}/{idx} -----------------')
            print(prompt)
            print(generated_text)
            print(f'----------------- end generated text -------------------')

print("final perplexiity : ",perp,"\n")
print("final Epoch :",num_epoch)


In [ ]:
print(complete2(model2, "print(", 512, sample=False))

In [ ]:
print(complete2(model2, "If ", 512, sample=False))

In [ ]:
print(complete2(model2, "while", 512, sample=False))

In [ ]:
print(complete2(model2, "for i in range", 512, sample=False))